# Data Proprecessing + Analysis
---
텍스트 데이터들을 전처리 하고 필요한 분석들을 함께 진행합니다.

- 배틀넷
   - 시간 데이터 전처리
   - 공지 글 삭제
   - 중복 데이터 전처리
   - 헤비유저 파악
   
  
- 인벤
   - 시간 데이터 전처리
   - 공지 글 삭제
   - contents컬럼의 NaN 제거
   - 헤비유저 파악
   
   
- 배틀넷+인벤 불용어 전처리

In [1]:
## 라이브러리 로딩

import pandas as pd # 판다스 
import numpy as np # 넘파이
import glob # 파일 여러개 불러올 때
import time, datetime
from datetime import datetime

import re # 정규표현식
import csv # 파일로 저장하기 위한 툴

import ast # Abstrac Syntex Tree - 스트링 처리된 리스트를 리스트로 다시 리턴할때

In [2]:
from tqdm import tqdm, trange
from tqdm import tqdm_notebook
from time import sleep

### - 인벤 데이터 올리기

In [3]:
# Inven Data 

df1 = pd.read_csv("data/invenData_1~10000.csv", encoding = 'utf8', header=None, engine='python')
df2 = pd.read_csv("data/invenData_10001~50000.csv", encoding='utf8', header=None, engine='python')
df3 = pd.read_csv("data/invenData_50001~70000.csv", encoding='utf8', header=None, engine='python')
df4 = pd.read_csv("data/invenData_70001~85834.csv", encoding='utf8', header=None, engine='python')
df5 = pd.read_csv("data/invenData_85835~100000.csv", encoding='utf8', header=None, engine='python')
df6 = pd.read_csv("data/invenData_100001~120000.csv", encoding='utf8', header=None, engine='python')
df7 = pd.read_csv("data/invenData_120001~140000.csv", encoding='utf8', header=None, engine='python')
df8 = pd.read_csv("data/invenData_140001~152823.csv", encoding='utf8', header=None, engine='python')
df9 = pd.read_csv("data/invenData_152824~160100.csv", encoding='utf8', header=None, engine='python')

In [4]:
# 인벤 데이터 한꺼번에 합침
df_inven = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])

In [5]:
# 컬럼명 붙이기
df_inven.columns = ['link_id', 'timestamp', 'title', 'contents']

In [6]:
# NaN을 아예 길이없는 스트링으로 전환
df_inven['contents'].fillna("", inplace=True)

In [7]:
## 내용이 너무 짧다. 제목과 내용을 합친다.

# 시작 컬럼 포지션을 잡기
source_col_loc = df_inven.columns.get_loc('title') 

# 컬럼 기준으로 제목 + 내용 합치기
df_inven['contents_all'] = df_inven.iloc[:,source_col_loc:source_col_loc+2].apply(lambda x: ",".join(x.astype(str)), axis=1)

In [10]:
df_inven.drop('contents_all',axis=1, inplace=True)

In [8]:
# 인덱스를 리셋시켜서 14만번대가 나오게 하도록 한다.
df_inven.reset_index(inplace=True)

In [42]:
df_inven[df_inven['title']=='놀러오세요 그아조씨의 "시공"방송에 [트/유]']

,index,link_id,timestamp,title,contents
1892,1892,1213066,2018-08-16 00:53,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
2948,2948,1211894,2018-08-02 23:03,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
4116,4116,1210601,2018-07-23 23:44,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
4726,4726,1209901,2018-07-18 01:12,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
6130,6130,1208347,2018-07-03 23:49,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
7855,7855,1206425,2018-06-14 22:48,"놀러오세요 그아조씨의 ""시공""방송에 [트/유]",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다. ...


In [41]:
df_inven[df_inven['title']=='[트/유]놀러오세요 그아조씨의 "시공"방송에']

,index,link_id,timestamp,title,contents
6445,6445,1207995,2018-06-29 23:06,"[트/유]놀러오세요 그아조씨의 ""시공""방송에",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
6789,6789,1207603,2018-06-26 01:01,"[트/유]놀러오세요 그아조씨의 ""시공""방송에",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...
7099,7099,1207249,2018-06-21 21:34,"[트/유]놀러오세요 그아조씨의 ""시공""방송에",안녕하세요~ 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바드 스피오테입니다.오늘...


In [38]:
df_inven[df_inven['title']=='안녕하세요.트위치/유튜브에서 시공의 폭풍 방송합니다 ^^']

,index,link_id,timestamp,title,contents
13301,3323,1199990,2018-05-25 23:49,안녕하세요.트위치/유튜브에서 시공의 폭풍 방송합니다 ^^,안녕하세요~ 다시 인사 드립니다. 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바...
14087,4109,1199088,2018-05-23 21:53,안녕하세요.트위치/유튜브에서 시공의 폭풍 방송합니다 ^^,안녕하세요~ 다시 인사 드립니다. 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바...
14487,4509,1198616,2018-05-22 23:10,안녕하세요.트위치/유튜브에서 시공의 폭풍 방송합니다 ^^,안녕하세요~ 다시 인사 드립니다. 종합 게임 방송을 지향하고 있는 하꼬 스트리머 바...


In [37]:
df_inven.head()

,index,link_id,timestamp,title,contents
0,0,1214270,2018-08-30 16:06,댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다 보면 때로는 나와 조금 다른 의견...
1,1,1214678,2018-09-05 11:56,9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 스톰 인벤 커뮤니티팀입니다.히어로즈 인벤에서는 매주 ...
2,2,1214899,2018-09-06 22:38,"[오시공] 김정민,문규리,침착맨,네클릿,옥냥이 메피스토 특별방송!",
3,3,1215180,2018-09-10 23:57,ㅋㅋㅋㅋ 장구벌레,
4,4,1215179,2018-09-10 23:52,캘타스 뭐가 더 비슷한가요?,타겜이지만 ㅎ.. 고민


### - 인벤
---
### 시간 데이터 전처리

In [36]:
df_inven.head()

,index,link_id,timestamp,title,contents,contents_all
0,0,1214270,2018-08-30 16:06,댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다 보면 때로는 나와 조금 다른 의견...,"댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다..."
1,1,1214678,2018-09-05 11:56,9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 스톰 인벤 커뮤니티팀입니다.히어로즈 인벤에서는 매주 ...,"9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ..."
2,2,1214899,2018-09-06 22:38,"[오시공] 김정민,문규리,침착맨,네클릿,옥냥이 메피스토 특별방송!",,"[오시공] 김정민,문규리,침착맨,네클릿,옥냥이 메피스토 특별방송!,"
3,3,1215180,2018-09-10 23:57,ㅋㅋㅋㅋ 장구벌레,,"ㅋㅋㅋㅋ 장구벌레,"
4,4,1215179,2018-09-10 23:52,캘타스 뭐가 더 비슷한가요?,타겜이지만 ㅎ.. 고민,"캘타스 뭐가 더 비슷한가요?,타겜이지만 ㅎ.. 고민"


In [37]:
# 새로운 데이트타임 형식 만들기

inven_new_dates = []

for i in range(len(df_inven)):
 
    inven_dates = datetime.strptime(df_inven.iloc[i,2], "%Y-%m-%d %H:%M")
    inven_new_dates.append(inven_dates)

In [38]:
# 시간 열을 추가하고 없앱니다.
df_inven["new_datetime"] = inven_new_dates
df_inven.drop('timestamp',axis=1, inplace=True)

In [42]:
# 분석에 필요없어진 열들을 다시 드랍합니다.

#df_inven.drop('title',axis=1, inplace=True)
#df_inven.drop('contents',axis=1, inplace=True)
df_inven.drop('index',axis=1, inplace=True)

In [43]:
df_inven.tail()

,link_id,contents_all,new_datetime
147578,1030725,"와 미친 오늘 진짜 치열하다 ㅋㅋㅋ,5,6 위 매치 진짜 미친듯이 치열하네의외의 꿀...",2017-09-02 19:19:00
147579,1030724,"뭐 안늙어죽으면 불멸자는 맞지만,불멸자들중에서도 조건부로맞아죽어도 살아날수있으니 개...",2017-09-02 19:18:00
147580,1030723,"히오스 n주년 이벤 해서,플레이어들이스킨 부문영웅 부문 창작해서 각 서버별로 상금걸...",2017-09-02 19:16:00
147581,1030722,"Cc에 강한 탱커 생겻으면,Cc 맞으면 체력회복",2017-09-02 19:13:00
147582,1030721,"이겜 영리하다보면 어느구간부터가 잘하는지 의문임,체감상 그마도 몇십유저 빼고 다 고...",2017-09-02 19:13:00


### 인벤 - 그 외 중복 데이터 처리하기

In [44]:
print(df_inven[df_inven["link_id"] == 1214678],"\n","="*80,
      df_inven[df_inven["link_id"] == 1214270],"\n","="*80,
      df_inven[df_inven["link_id"] == 1213632],"\n","="*80,
      df_inven[df_inven["link_id"] == 1214199],"\n","="*80,
      df_inven[df_inven["link_id"] == 1213035],"\n","="*80,
      df_inven[df_inven["link_id"] == 1212385],"\n","="*80,
      df_inven[df_inven["link_id"] == 1211671],"\n","="*80,
      df_inven[df_inven["link_id"] == 1210740],"\n","="*80,
      df_inven[df_inven["link_id"] == 1209936],"\n","="*80,
      df_inven[df_inven["link_id"] == 1208994],"\n","="*80,
      df_inven[df_inven["link_id"] == 1208372],"\n","="*80,
      df_inven[df_inven["link_id"] == 1207739],"\n","="*80,
      df_inven[df_inven["link_id"] == 1207110],"\n","="*80)

     link_id                                       contents_all  \
1    1214678  9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   
460  1214678  9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   

           new_datetime  
1   2018-09-05 11:56:00  
460 2018-09-05 11:56:00   
 ================================================================================      link_id                                       contents_all  \
0    1214270  댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다...   
821  1214270  댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다...   

           new_datetime  
0   2018-08-30 16:06:00  
821 2018-08-30 16:06:00   
 ================================================================================       link_id                                       contents_all  \
1390  1213632  8월 3주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   

            new_datetime  
1390 2018-08-22 16:38:00   
 ================================================================================ 

1, 460 / 0 ,821 / 1390, 881, 1921, 2510, 3140, 3990, 4695, 5531, 6109, 6672, 7226  글들은 커뮤니티 팀에서 올린 것으로 삭제한다.

(운영자 아이디로 검색하여 글번호 확인하였음)

In [46]:
# 운영팀에서 올린 공지는 제외
# 아이디를 직접 긁어오지 않았으므로, 웹상에서 직접 링크 아이디를 찾아 제거합니다.

df_inven.drop([0],inplace=True)
df_inven.drop([1],inplace=True)
df_inven.drop([460],inplace=True)
df_inven.drop([821],inplace=True)
df_inven.drop([881],inplace=True)
df_inven.drop([1390],inplace=True)
df_inven.drop([1921],inplace=True)
df_inven.drop([2510],inplace=True)
df_inven.drop([3140],inplace=True)
df_inven.drop([3990],inplace=True)
df_inven.drop([4695],inplace=True)
df_inven.drop([5531],inplace=True)
df_inven.drop([6109],inplace=True)
df_inven.drop([6672],inplace=True)
df_inven.drop([7226],inplace=True)


# 기타 중복글



In [47]:
df_inven.reset_index(inplace=True)

In [49]:
df_inven.drop('index',axis=1, inplace=True)

In [50]:
# inven 컬럼 다시조정

df_inven = df_inven[['link_id','new_datetime','contents_all']]

In [64]:
# 년도와 월을 따로 만듭니다.

df_inven['Year'] = df_inven['new_datetime'].dt.year
df_inven['Month'] = df_inven['new_datetime'].dt.month


In [65]:
df_inven.head()

,link_id,new_datetime,contents_all,Year,Month
0,1214899,2018-09-06 22:38:00,오시공 김정민 문규리 침착맨 네클릿 옥냥이 메피스토 특별방송,2018,9
1,1215180,2018-09-10 23:57:00,ㅋㅋㅋㅋ 장구벌레,2018,9
2,1215179,2018-09-10 23:52:00,캘타스 뭐가 더 비슷한가요 타겜이지만 ㅎ 고민,2018,9
3,1215178,2018-09-10 23:52:00,메피 방금사서 해봤는데 너무사기임 궁각만 잘재고 쓰면 못해도 더블킬은 따고들...,2018,9
4,1215177,2018-09-10 23:50:00,제발 메피는 궁이랑 보호막특 너프 좀 두개가 안되면 궁이라도 궁자체도 최상...,2018,9


## 3. 데이터 전처리 - 특수문자 전처리

In [55]:
# 정규 표현식
p_http = re.compile("(((http(s?))\:\/\/)?)([0-9a-zA-Z\-]+\.)+[a-zA-Z]{2,6}(\:[0-9]+)?(\S*)")
p_char = re.compile("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9ㅣ ]")
p_tag = re.compile("<.+>")
p_brac = re.compile("[[\]]")
p_hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 

In [56]:
# 정규 표현식 함수화

def patternRemover(doc):
    doc = p_http.sub(' ', doc)
    doc = p_char.sub(' ', doc)
    doc = p_tag.sub(' ', doc)
    doc = p_brac.sub(' ', doc)
    doc = p_hangu.sub(' ', doc)
    return doc

In [58]:
# 배틀넷 - 특수문자 전처리

for i in range(len(df_battle["contents_all"])):
    df_battle.iloc[i,2] = patternRemover(df_battle.iloc[i,2])

In [66]:
len(df_inven)

147568

In [70]:
# 인벤 - 특수문자 전처리

%%time

total = tqdm_notebook(range(147568))

for i in total:
    df_inven.iloc[i,2] = patternRemover(df_inven.iloc[i,2])

CPU times: user 36min 53s, sys: 1min 28s, total: 38min 22s
Wall time: 39min 26s


In [73]:
df_inven.iloc[140000,2]

## 초성 나왔음 초성도 제거해야해..

'그래서 이스프레이는 언제나오죠 나오면 바로삼 ㄹㅇ'

### 데이터 합치기

In [75]:
df_all = pd.concat([df_battle, df_inven])

In [77]:
df_all.reset_index(inplace=True)

In [81]:
df_all.drop('index',axis=1, inplace=True)

In [163]:
df_all.to_csv("predata.csv", encoding='utf-8')

In [160]:
df_all.drop('tokens_contents', axis=1, inplace=True)

In [162]:
df_all.tail()

,link_id,new_datetime,contents_all,Year,Month
153203,1030725,2017-09-02 19:19:00,와 미친 오늘 진짜 치열하다 ㅋㅋㅋ 5 6 위 매치 진짜 미친듯이 치열하네의외의 꿀...,2017,9
153204,1030724,2017-09-02 19:18:00,뭐 안늙어죽으면 불멸자는 맞지만 불멸자들중에서도 조건부로맞아죽어도 살아날수있으니 개...,2017,9
153205,1030723,2017-09-02 19:16:00,히오스 n주년 이벤 해서 플레이어들이스킨 부문영웅 부문 창작해서 각 서버별로 상금걸...,2017,9
153206,1030722,2017-09-02 19:13:00,Cc에 강한 탱커 생겻으면 Cc 맞으면 체력회복,2017,9
153207,1030721,2017-09-02 19:13:00,이겜 영리하다보면 어느구간부터가 잘하는지 의문임 체감상 그마도 몇십유저 빼고 다 고...,2017,9
